In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
## Load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
### Drop irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.sample(6)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6152,508,France,Female,64,2,0.00,1,1,1,6076.62,0
7971,633,Spain,Male,42,10,0.00,1,0,1,79408.17,0
6045,549,Germany,Female,37,1,130622.34,2,1,1,128499.94,0
9641,680,France,Male,34,9,0.00,2,1,1,95686.60,0
1597,771,France,Male,32,5,62321.62,1,1,1,40920.59,0
937,850,Germany,Female,44,5,128605.32,1,0,1,171096.20,0


In [4]:
## Encode categorical variables
le_gender=LabelEncoder()
data['Gender']=le_gender.fit_transform(data['Gender'])
data.sample(6)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6703,681,Germany,1,49,8,142946.18,1,0,0,187280.51,1
9220,598,France,1,60,4,0.00,1,1,0,197727.14,1
9267,748,France,1,32,5,154737.88,2,1,1,172638.13,0
7267,507,Germany,1,31,9,111589.67,1,1,0,150037.19,0
5049,784,Spain,1,38,10,122267.85,1,0,0,145759.93,0
9200,739,France,0,30,6,0.00,1,0,0,122604.44,0


In [5]:
## Onehot encode 'Geography
ohe_geo=OneHotEncoder()
geo_encoder=ohe_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
ohe_features = ohe_geo.get_feature_names_out(['Geography'])

In [7]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=ohe_features)
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [9]:
data.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
7974,716,1,50,2,119655.77,1,1,1,12944.17,1,0.0,1.0,0.0
1584,671,0,42,3,0.00,2,1,1,128449.33,0,0.0,0.0,1.0
1641,712,1,53,2,111061.01,2,0,0,26542.17,0,0.0,0.0,1.0
2118,460,0,27,7,0.00,2,1,0,156150.08,1,1.0,0.0,0.0
3873,530,1,37,4,0.00,2,1,1,164844.37,0,0.0,0.0,1.0


In [15]:
## Save the encoders and sscaler
with open('le_gender.pkl','wb') as file:
    pickle.dump(le_gender,file)

with open('ohe_geo.pkl','wb') as file:
    pickle.dump(ohe_geo,file)


In [10]:
## DiVide the dataset into indepent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [18]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [19]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Implementation

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [16]:
node = X_train.shape[1]

In [17]:
## Build Our ANN Model
model=Sequential([
    Dense(node*5,activation='relu',input_shape=(node,)), ## HL1 Connected wwith input layer
    Dense(node*3,activation='relu'), ## HL2
    Dense(node*1,activation='relu'), ## HL3
    Dense(1,activation='sigmoid')  ## output layer
])

In [18]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 60)             │           780 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 36)             │         2,196 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 12)             │           444 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,433 (13.41 KB)

 Trainable params: 3,433 (13.41 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
loss=tf.keras.losses.BinaryCrossentropy()

In [20]:
## compile the model
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [21]:
## Set up the Tensorboard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [22]:
## Set up Early Stopping
es=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [23]:
### Train the model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                  callbacks=[tensorflow_callback,es])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7940 - loss: 0.4906 - val_accuracy: 0.8400 - val_loss: 0.3848
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8402 - loss: 0.3893 - val_accuracy: 0.8580 - val_loss: 0.3492
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8598 - loss: 0.3459 - val_accuracy: 0.8595 - val_loss: 0.3479
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8664 - loss: 0.3286 - val_accuracy: 0.8560 - val_loss: 0.3518
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8648 - loss: 0.3300 - val_accuracy: 0.8575 - val_loss: 0.3431
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8664 - loss: 0.3217 - val_accuracy: 0.8570 - val_loss: 0.3502
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8648 - loss: 0.3255 - val_accuracy: 0.8565 - val_loss: 0.3400
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8600 - loss: 0.3240 - val_accu

In [24]:
model.save('Churn.h5')

In [27]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [28]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 4796), started 0:02:03 ago. (Use '!kill 4796' to kill it.)

In [ ]:
### Load the pickle file
